In [ ]:
import numpy as np
import torch
%matplotlib inline
import matplotlib.pyplot as plt
from torch import nn, optim, autograd
from torch.nn import functional as F
from pyDOE import lhs
import scipy.io

#Paper reproduction
torch.manual_seed(1234)
torch.cuda.manual_seed(1234)
np.random.seed(1234)
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.gridspec as gridspec
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [ ]:
def weights_init(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_normal_(m.weight)
        nn.init.constant_(m.bias, 0)
    elif isinstance(m, nn.Conv2d):
        #nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
        nn.init.xavier_normal_(m.weight)
        nn.init.constant_(m.bias, 0)
    elif isinstance(m, nn.BatchNorm2d):
        nn.init.constant_(m.weight, 1)
        nn.init.constant_(m.bias, 0)

In [ ]:
class hidden_layers(nn.Module):
    def __init__(self,input_number,output_number):
        super(hidden_layers, self).__init__()
        self.layer = nn.Linear(input_number,output_number)
    def forward(self, x):
        x = self.layer(x)
        x = torch.sin(x)
        return x

In [ ]:
class NN_H1 (nn.Module):
    def __init__(self,in_N, width, depth, out_N):
        #depth = layers-2
        super(NN_H1, self).__init__()
        self.in_N = in_N
        self.width = width
        self.depth = depth
        self.out_N = out_N

        self.stack = nn.ModuleList()

        self.stack.append(nn.Linear(in_N, width))

        for i in range(depth):
            self.stack.append(nn.Linear(width, width))

        self.stack.append(nn.Linear(width, out_N))
        
        
    def forward(self, x):
        for m in self.stack:
            x = m(x)
        return x

In [ ]:
class NN_H2 (nn.Module):
    def __init__(self,in_N, width, depth, out_N):
        #depth = layers-2
        super(NN_H2, self).__init__()
        self.in_N = in_N
        self.width = width
        self.depth = depth
        self.out_N = out_N

        self.stack = nn.ModuleList()

        self.stack.append(hidden_layers(in_N, width))

        for i in range(depth):
            self.stack.append(hidden_layers(width, width))

        self.stack.append(nn.Linear(width, out_N))
        
        
    def forward(self, x):
        for m in self.stack:
            x = m(x)
        return x

In [ ]:
class get_discriminator(nn.Module):
    def __init__(self,in_N, width, depth, out_N):
        #depth = layers-2
        super(get_discriminator, self).__init__()
        self.in_N = in_N
        self.width = width
        self.depth = depth
        self.out_N = out_N

        self.stack = nn.ModuleList()

        self.stack.append(hidden_layers(in_N, width))

        for i in range(depth):
            self.stack.append(hidden_layers(width, width))

        self.stack.append(nn.Linear(width, out_N))
        
        
    def forward(self, x):
        for m in self.stack:
            x = m(x)
        x = torch.sigmoid(x)
        return x 

# Data For PINNs

In [ ]:

N_train = 5000
N_bound = 100

np.random.seed(12345)
X_train = np.random.uniform(0,1,(N_train,2))
X_bound_zero_one = np.concatenate((np.zeros((int(N_bound/4),1)),np.ones((int(N_bound/4),1))),0)
np.random.seed(123456)
X_bound1 = np.concatenate((np.random.uniform(0,1,(int(N_bound/2),1)),X_bound_zero_one),1)
np.random.seed(1234567)
X_bound2 = np.concatenate((X_bound_zero_one,np.random.uniform(0,1,(int(N_bound/2),1))),1)

X_bound = np.concatenate((X_bound1,X_bound2),0)

In [ ]:
#x,t  #u,v
X_train = torch.from_numpy(X_train).float()
X_bound = torch.from_numpy(X_bound).float()
x = X_train[:,0:1]
t = X_train[:,1:2]

x.requires_grad_()
t.requires_grad_()

### Labeled Data

In [ ]:
n = 5
np.random.seed(1111)
gan_data_x_t = lhs(2,n)
gan_data_u = 1/(2*np.pi**2)*np.sin(gan_data_x_t[:,0:1]*np.pi)*np.sin(gan_data_x_t[:,1:2]*np.pi)
gan_data_x_t = torch.from_numpy(gan_data_x_t).float()
#gan_data_u = torch.sin(k*gan_data_x_t)
gan_data_u = torch.from_numpy(gan_data_u).float()

### The Image With Labeled Sample

In [ ]:
traindata = np.concatenate((np.linspace(0,1,200).reshape(-1,1),np.linspace(0,1,200).reshape(-1,1)),1)
x_ = traindata[:,0:1]
y_ = traindata[:,1:2]
xx,yy = np.meshgrid(x_,y_)
data_numpy = np.concatenate((xx.reshape(-1,1),yy.reshape(-1,1)),1)

aa = 1/(2*np.pi**2)*np.sin(data_numpy[:,0]*np.pi)*np.sin(data_numpy[:,1]*np.pi)
plt.imshow(aa.reshape(200,200), interpolation='nearest',extent=[0, 1, 0, 1], cmap='rainbow')#其实可以不加origin参数 对于这组数据相当于origin='upper'，其实就是按照数据原来的位置画图
plt.colorbar(shrink=.98)
plt.scatter(gan_data_x_t[:,0:1], gan_data_x_t[:,1:2],c='k',marker = 'x',alpha=1,s=15)
plt.savefig('Possion h points.eps',format='eps',dpi=1000, bbox_inches = 'tight')

plt.show()

In [ ]:
def relative_l2(u_pred,u_real):
    l2 = np.linalg.norm(u_real-u_pred,2)/np.linalg.norm(u_real,2)
    return l2

# Test Data

In [ ]:
np.random.seed(5678)
test_data = lhs(2,5000)
test_data_tensor = torch.from_numpy(test_data).float()
aa = 1/(2*np.pi**2)*np.sin(test_data[:,0]*np.pi)*np.sin(test_data[:,1]*np.pi)

# Method2
**$\text { GA - PINN }$**

<div class="alert alert-info">`loss_function：`

$\mathrm{L}_D=\frac{1}{J} \sum_{j=1}^J\left(1-D\left[\left(\mathbf{x}_T^{(j)}, u_T^{(j)}\right)\right]\right)+D\left[\left(x_L^{(j)}, G\left[x_L^{(j)}\right]\right)\right] \\
\mathrm{L}_G=\mathrm{L}_T+\frac{1}{J} \sum_{j=1}^J\left(1-D\left[\left(x_T^{(j)}, G\left[x_T^{(j)}\right]\right)\right]\right)\\
\mathrm{L}_{\text {PINN }}:=\mathrm{L}_f+\lambda_1 \mathrm{~L}_b \text { with } \mathrm{L}_b:=\sum_{i=1}^I \mathrm{~L}_{b_i}
$



</div>

In [ ]:
torch.manual_seed(1234)
torch.cuda.manual_seed(1234)
np.random.seed(1234)

PINNs2 = NN_H2(2, 100, 4, 1)
PINNs2.apply(weights_init)
optimizer1 = optim.Adam([{'params': PINNs2.parameters()}], lr=1e-3)


discriminator= get_discriminator(3, 100, 1, 1)
discriminator.apply(weights_init)
optimizer2 = optim.Adam([{'params': discriminator.parameters(), 'weight_decay': 0.01}], lr=5e-3)

loss_all_2= []
test_loss_2 = []

#########gpu############
discriminator.cuda()
X_bound = X_bound.cuda()
gan_data_x_t = gan_data_x_t.cuda()
gan_data_u = gan_data_u.cuda()
PINNs2.cuda()
E_bound = PINNs2(X_bound)
real_bound = torch.zeros_like(E_bound)
real_bound = real_bound.cuda()
x = x.cuda()
t = t.cuda()
#########gpu############


loss1_value = 1
it = 0

while loss1_value > 5e-5 :
    PINNs2.cuda()
    
    ##############loss D############
    pre_H = PINNs2(gan_data_x_t)
    d_fake = discriminator(torch.cat((gan_data_x_t,pre_H.detach()),1))
    d_real = discriminator(torch.cat((gan_data_x_t,gan_data_u),1))
    
    loss_d = torch.mean(1-d_real)+torch.mean(d_fake)
    
    optimizer2.zero_grad()
    loss_d .backward()
    optimizer2.step()  
    

    
    ##############loss G ############
    for param_group in optimizer1.param_groups:
        param_group["lr"]=1e-6
    pre_H = PINNs2(gan_data_x_t)
    d_fake = discriminator(torch.cat((gan_data_x_t,pre_H.detach()),1))
    loss_L = torch.mean(torch.square(pre_H - gan_data_u))+torch.mean(1-d_fake)
    
    optimizer1.zero_grad()
    loss_L.backward()
    optimizer1.step()  
    
    
    
    for param_group in optimizer1.param_groups:
        param_group["lr"]=1e-3 
    ##### loss_Bi  ######
    E_bound = PINNs2(X_bound)
    real_bound = torch.zeros_like(E_bound)
    loss_bound = torch.mean(torch.square(E_bound-real_bound))
    
    ##### loss f  ######
    
    E_inside = PINNs2(torch.cat((x,t),1))
    E_x = autograd.grad(outputs=E_inside, inputs=x,
                              grad_outputs=torch.ones_like(E_inside),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]
    E_xx = autograd.grad(outputs=E_x, inputs=x,
                              grad_outputs=torch.ones_like(E_x),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]
    E_t = autograd.grad(outputs=E_inside, inputs=t,
                              grad_outputs=torch.ones_like(E_inside),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]
    E_tt = autograd.grad(outputs=E_t, inputs=t,
                              grad_outputs=torch.ones_like(E_t),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]       
    deata_E = E_xx+E_tt
    loss_f = torch.mean(torch.square(deata_E+torch.sin(torch.pi*x)*torch.sin(torch.pi*t)))
    
    pre_H = PINNs2(gan_data_x_t)
    

    loss_g = loss_f+loss_bound
    
    loss_p = loss_f+loss_bound
    loss1_value = loss_p.item()

    loss_all_2.append(loss1_value)
    
    optimizer1.zero_grad()
    loss_g.backward()
    optimizer1.step()
    
  
    
    #########  test_loss NRMSE  #########
    PINNs2.cpu()
    test_loss = relative_l2(PINNs2(test_data_tensor).detach().numpy(),aa.reshape(-1,1))
    test_loss_2.append(test_loss)
    
    
    if it % 100 == 0:
        print('It:', it, 'Loss:', loss_p.item())
    it = it + 1         
loss_p.item()

# Results

## Save data

In [ ]:
# np.save('../experimental_data/method_2/test_loss_2',test_loss_2)
# np.save('../experimental_data/method_2/loss_all_2',loss_all_2)
# torch.save(PINNs2,'../saved_model/PINNs2')

## Epoch and NRMSE

In [ ]:
Epochs = len(test_loss_2)
NRMSE = relative_l2(PINNs2(test_data_tensor).detach().numpy(),aa.reshape(-1,1))

print('Epochs:',Epochs,'NRMSE:',NRMSE)